# Benchmark
This notebook shows the benchmark results of all covered ops.

## Setup

In [1]:
from IPython.display import clear_output

def build_package(name, url, commit=None, deps=True):
    import importlib
    import os, sys
    if importlib.util.find_spec(name) is None:
        os.system(f"git clone {url} {name} || true")
        if commit is not None:
            os.system(f"cd {name}; git checkout {commit}")
        os.system(f"cd {name}; git submodule update --init --recursive")
        no_deps = ""
        if deps:
            os.system(f"cd {name}; pip3 install -r requirements.txt || true")
        else:
            no_deps = "--no-deps"
        clear_output()
        os.system(f'cd {name}; pip3 install -e ".[dev]" {no_deps}')

build_package("transformers", "https://github.com/huggingface/transformers.git", deps=False)
build_package("xformers", "https://github.com/facebookresearch/xformers.git")
build_package("epoi", "https://github.com/comaniac/epoi.git")

## Benchmark

In [2]:
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

def load_ipython_extension():
    display(Javascript(disable_js))
    print ("autoscrolling long output is disabled")
    
load_ipython_extension()

<IPython.core.display.Javascript object>

autoscrolling long output is disabled


In [3]:
!python3 -m epoi.benchmark

===== Environment =====

GPU: Tesla V100-SXM2-16GB

PyTorch Configuration
   Config         Value
-------------  ------------
   Version     1.13.0+cu117
Built w. CUDA      11.7


Other Libraries Configuration
  Package       Version                   Commit SHA
------------  -----------  ----------------------------------------
    epoi        0.1.dev    c1a64b8a13542174dfe74dad628ea26ce57d7fc1
transformers  4.24.0.dev0  12ce2941c7b67c0dedac0f0468b3ed854fa940ab
  xformers    0.0.14.dev   ba93c5012d00bd1b010514a7bc9bd938c1ad6149
   triton        2.0.0                       N/A
    apex          0.1                        N/A
===== Environment =====

[2022-10-28 19:54:50] INFO main: Selected bias_gelu
[2022-10-28 19:54:50] INFO main: Selected dropout_add_ln
[2022-10-28 19:54:50] INFO main: Selected bert_attention
[2022-10-28 19:54:50] INFO main: Selected gpt_attention
[2022-10-28 19:54:50] INFO main: Selected qkv_self_attn
[2022-10-28 19:54:50] INFO main: Selected layer_norm
[2022-10-28

[2022-10-28 20:15:06] INFO bencher: Correctness checking for Apex (FP16) is passed
[2022-10-28 20:15:07] WARNING bencher: Skip correctness checking for Triton (FP16): Forward failed
[2022-10-28 20:15:07] INFO bencher: Correctness checking for xFormers (FP16) is passed
[2022-10-28 20:15:08] WARNING bencher: Skip Triton (FP16): Forward failed with shape (32, 128, 768)
[2022-10-28 20:15:10] WARNING bencher: Skip Triton (FP16): Forward failed with shape (8, 512, 1024)
[2022-10-28 20:15:18] WARNING bencher: Skip Triton (FP16): Forward failed with shape (16, 512, 8192)
[2022-10-28 20:15:27] WARNING bencher: Skip Triton (FP16): Forward failed with shape (4, 2048, 8192)
[------------------------------------------------- LayerNorm -------------------------------------------------]
                       |  PyTorch (FP32)  |  Apex (FP32)  |  PyTorch (FP16)  |  Apex (FP16)  |  xFormers (FP16)
1 threads: ----------------------------------------------------------------------------------------------